In [ ]:
!pip install "git+https://github.com/aresPanos/Interpretable-Point-Processes.git"
!pip install torch
!pip install --ignore-installed llvmlite
!pip install pomegranate
!pip install scipy

import time
import numpy as np
import argparse
import os
import pandas as pd
from vi_dpp import VI_model
import pickle
from sklearn.metrics import f1_score, accuracy_score
from sklearn.metrics import f1_score
from pomegranate import *
from scipy.stats import lognorm
import torch
import time
import sys
from sklearn.metrics import accuracy_score
from sklearn.mixture import GaussianMixture
from scipy.special import logsumexp
import warnings
import json
warnings.filterwarnings("ignore")
print('Done')

The Section Below imports the necessary data

In [ ]:
#2022 Data
with open("timestamps_2022_test.json", "r") as file:
    my_list = json.load(file)
type(my_list)
times_all = [np.array(value) for value in my_list.values()]
type(times_all)
print('Event Times:')
#print(times_all[:5])


with open("marklist_2022_test.json", "r") as file:
    my_list = json.load(file)
type(my_list)
marks_all = [np.array(value) for value in my_list.values()]
type(marks_all)
print('Mark list:')
#print(marks_all[:5])

with open("teamlist_2022_test.json", "r") as file:
    my_list = json.load(file)
type(my_list)
teams_all = [np.array(value) for value in my_list.values()]
type(teams_all)
print('Teams list:')
#print(teams_all[:5])

with open("zonelist_2022_test.json", "r") as file:
    my_list = json.load(file)
type(my_list)
zones_all = [np.array(value) for value in my_list.values()]
type(zones_all)
print('Zones list:')
#print(zones_all[:5])

with open("timestamps_2023_final.json", "r") as file:
    my_list = json.load(file)
type(my_list)
times_all_2023 = [np.array(value) for value in my_list.values()]
type(times_all)
print('Event Times:')
#print(times_all[:5])

#2023 Data
with open("marklist_2023_final.json", "r") as file:
    my_list = json.load(file)
type(my_list)
marks_all_2023 = [np.array(value) for value in my_list.values()]
type(marks_all)
print('Mark list:')
#print(marks_all[:5])

with open("teamlist_2023_final.json", "r") as file:
    my_list = json.load(file)
type(my_list)
teams_all_2023 = [np.array(value) for value in my_list.values()]
type(teams_all)
print('Teams list:')
#print(teams_all[:5])

with open("zonelist_2023_final.json", "r") as file:
    my_list = json.load(file)
type(my_list)
zones_all_2023 = [np.array(value) for value in my_list.values()]
type(zones_all)
print('Zones list:')
#print(zones_all[:5])

#2022 Data Rebounds Removed
with open("noreb_timestamps_2022_final.json", "r") as file:
    my_list = json.load(file)
type(my_list)
noreb_times_all_2022 = [np.array(value) for value in my_list.values()]
type(times_all)
print('Event Times:')
#print(times_all[:5])


with open("noreb_marklist_2022_final.json", "r") as file:
    my_list = json.load(file)
type(my_list)
noreb_marks_all_2022 = [np.array(value) for value in my_list.values()]
type(marks_all)
print('Mark list:')
#print(marks_all[:5])

with open("noreb_teamlist_2022_final.json", "r") as file:
    my_list = json.load(file)
type(my_list)
noreb_teams_all_2022 = [np.array(value) for value in my_list.values()]
type(teams_all)
print('Teams list:')
#print(teams_all[:5])

with open("noreb_zonelist_2022_final.json", "r") as file:
    my_list = json.load(file)
type(my_list)
noreb_zones_all_2022 = [np.array(value) for value in my_list.values()]
type(zones_all)
print('Zones list:')
#print(zones_all[:5])

#2023 Data Rebounds Removed
with open("noreb_timestamps_2023_final.json", "r") as file:
    my_list = json.load(file)
type(my_list)
noreb_times_all_2023 = [np.array(value) for value in my_list.values()]
type(times_all)
print('Event Times:')
#print(times_all[:5])


with open("noreb_marklist_2023_final.json", "r") as file:
    my_list = json.load(file)
type(my_list)
noreb_marks_all_2023 = [np.array(value) for value in my_list.values()]
type(marks_all)
print('Mark list:')
#print(marks_all[:5])

with open("noreb_teamlist_2023_final.json", "r") as file:
    my_list = json.load(file)
type(my_list)
noreb_teams_all_2023 = [np.array(value) for value in my_list.values()]
type(teams_all)
print('Teams list:')
#print(teams_all[:5])

with open("noreb_zonelist_2023_final.json", "r") as file:
    my_list = json.load(file)
type(my_list)
noreb_zones_all_2023 = [np.array(value) for value in my_list.values()]
type(zones_all)
print('Zones list:')
#print(zones_all[:5])

The following defines a function for the Training, Test Split

In [19]:
#Training, Valditation and Testing data:
seed=2105098
def test_train(times_all,marks_all,teams_all,zones_all):
    if len(marks_all)==(1230*4):
        train_size=80/82
        val_size=2/82
    else:
        train_size=38/40
        val_size=2/40
    
    N = len(times_all)
    if seed is not None:
        np.random.seed(seed)
    all_idx = np.arange(N)
    
    train_end = int(train_size * N)
    val_end = int((train_size + val_size) * N)
    
    train_idx = all_idx[:train_end]
    val_idx = all_idx[train_end:]
    #val_idx = all_idx[train_end:val_end]
    #test_idx = all_idx[val_end:]
    
    times_list_train, marks_list_train,teams_list_train,zones_list_train = [], [],[],[]
    times_list_val, marks_list_val,teams_list_val,zones_list_val = [], [],[],[]
    #times_list_test, marks_list_test,teams_list_test,zones_list_test = [], [],[],[]
    
    for idx in train_idx:
        times_list_train.append(times_all[idx])
        marks_list_train.append(marks_all[idx])
        teams_list_train.append(teams_all[idx])
        zones_list_train.append(zones_all[idx])
    
    for idx in val_idx:
        times_list_val.append(times_all[idx])
        marks_list_val.append(marks_all[idx])
        teams_list_val.append(teams_all[idx])
        zones_list_val.append(zones_all[idx])
    
    #for idx in test_idx:
       # times_list_test.append(times_all[idx])
       # marks_list_test.append(marks_all[idx])
       # teams_list_test.append(teams_all[idx])
       # zones_list_test.append(zones_all[idx])
    
    U_dim = max([mark.max() for mark in marks_all]) + 1
    num_teams = max([id_vec.max() for id_vec in teams_list_train]) +1
    
    #print(len(times_list_train),len(times_list_val),len(times_list_test))
    #print(len(marks_list_train),len(marks_list_val),len(marks_list_test))
    #print(len(teams_list_train),len(teams_list_val),len(teams_list_test))
    #print(len(zones_list_train),len(zones_list_val),len(zones_list_test))
    print('Length Of Training Data:',len(times_list_train))
    print('    Length Of Test Data:',len(times_list_val), '\n')


    return(times_list_train,marks_list_train,zones_list_train,teams_list_train,times_list_val,marks_list_val,zones_list_val,teams_list_val)

Here we have defined the VI Framework and functions for generating times, this is an adapted form of Panos 2023

In [21]:
def clamp_preserve_gradients(x: torch.Tensor, min: float, max: float) -> torch.Tensor:
    """Clamp the tensor while preserving gradients in the clamped region."""
    return x + (x.clamp(min, max) - x).detach()

def tick2myformat(timestamps):
    list_times = []
    list_marks = []

    N = len(timestamps)
    U = len(timestamps[0])

    for i in range(N):
        seq = timestamps[i]

        all_times_i = np.concatenate(seq)
        all_marks_i = []
        for u in range(U):
            all_marks_i = np.concatenate((all_marks_i, u*np.ones_like(seq[u], dtype=np.int) ))

        arg_ind = np.argsort(all_times_i)

        list_times.append(all_times_i[arg_ind])
        list_marks.append(all_marks_i[arg_ind].astype(np.int))

    return list_times, list_marks


class VI_Framework(object):
    def __init__(self,
                 data_times: list,
                 data_marks: list,
                 data_zones: list,
                 data_teamsID: list,
                 data_times_val: list,
                 data_marks_val: list,
                 data_zones_val: list,
                 data_teamsID_val: list,
                 cut_off: int,
                 num_samples: int=1,
                 verbose: bool=False,
                 tol=1e-5,
                 weight_temp=1.,
                 lr_gamma=0.9999,
                 max_iter: int=10000,
                 interval: int=100,
                 print_every: int=100,
                 offset: int=0,
                 momentum=0.5,
                lr_init = 0.03):

        if not (isinstance(data_times[0], np.ndarray) and isinstance(data_times_val[0], np.ndarray)):
            raise TypeError("Invalid 'data_times' or ''data_times_val' provided")

        if not (isinstance(data_marks[0], np.ndarray) and isinstance(data_marks_val[0], np.ndarray)):
            raise TypeError("Invalid 'data_marks' or 'data_marks_val' provided")

        if not (isinstance(data_zones[0], np.ndarray) and isinstance(data_zones_val[0], np.ndarray)):
            raise TypeError("Invalid 'data_zones' or'data_zones_val' provided")

        if not (isinstance(data_teamsID[0], np.ndarray) and isinstance(data_teamsID_val[0], np.ndarray)):
            raise TypeError("Invalid 'data_teamsID' or 'data_teamsID_val' provided")       

        self.tol = tol
        self.cut_off = cut_off
        self.lr_gamma = lr_gamma
        self.max_iter = max_iter
        self.num_samples = num_samples
        self.interval = interval
        self.print_every = print_every
        self.offset = offset
        self.momentum = momentum
        self.weight_temp = weight_temp
        self.verbose = verbose
        self.lr_init = lr_init

        self.U_dim = int(max([mark_vec.max() for mark_vec in data_marks]) + 1)
        self.HA_dim = int(self.U_dim/2)
        self.Z_dim = int(max([zone_vec.max() for zone_vec in data_zones]) + 1)
        self.num_teams = int(max([id_vec.max() for id_vec in data_teamsID]) + 1)
        self.size_alpha_betas = int(1 + self.Z_dim * self.U_dim**2)
        self.size_phis = int(self.Z_dim * self.U_dim * (self.U_dim - 1))
        self.size_deltas = int(self.Z_dim * self.U_dim)
        self.data_marks = data_marks
        self.data_zones = data_zones
        self.data_teamsID = data_teamsID
        self.data_times_val = data_times_val
        self.data_marks_val = data_marks_val
        self.data_zones_val = data_zones_val
        self.data_teamsID_val = data_teamsID_val
        self.num_seqs = len(data_marks)
        self.num_seqs_val = len(data_marks_val)
        self.num_jumps = [mark_vec.size for mark_vec in data_marks]
        self.num_jumps_val = [mark_vec.size for mark_vec in data_marks_val]
        self.elbo_history = np.zeros(self.max_iter)
        self.log_sqrt2pi = 0.5 * np.log(2 * np.pi)
        self.train_time = 0.
        self.u_b = 1 - 1e-6
        self.l_b = -1 + 1e-6
        self.num_gw = int(self.num_seqs/(self.num_teams*2))

        self.m_betas_ind = int(self.size_alpha_betas)
        self.sigma_betas_ind = int(2*self.size_alpha_betas)
        self.m_phis_ind = int(self.sigma_betas_ind + self.size_phis)
        self.sigma_phis_ind = int(self.m_phis_ind + self.size_phis)
        self.m_omega_all_ind = int(self.sigma_phis_ind + (self.U_dim-1) * self.num_teams)
        self.sigma_omega_all_ind = int(self.m_omega_all_ind + (self.HA_dim) * self.num_teams)
        self.rho_ind = int(self.sigma_omega_all_ind + (self.HA_dim) * self.num_teams)
        self.omegas_all_ind = int(self.rho_ind + (self.U_dim-1) * self.num_teams * self.num_gw)

        self.n_var_params = int(2 * (self.size_alpha_betas + self.size_phis) + self.size_deltas + self.num_teams * ((self.U_dim-1) + (self.HA_dim) + (self.HA_dim) + (self.U_dim-1) * self.num_gw))

        self.cache_data(data_times)

        self.positive_constraint = torch.nn.Softplus()
        self.minus_one_one_constraint = torch.nn.Sigmoid()
        self.positive_constraint_np = lambda x: np.log(1. + np.exp(x))
        self.coeffs = None
        self.coeffs_old = None
        self.prior_sigma_betas_alpha = torch.tensor(0.1 * np.ones(self.size_alpha_betas))


    def cache_data(self, data_times):
        self.tau_ij_minus_list = []
        for i in range(self.num_seqs):
            vec_times = data_times[i]
            N = vec_times.size
            minus_tau_ij = np.zeros((self.cut_off, N - 1))
            for q in range(self.cut_off):
                minus_tau_ij[q, q:N-1] = vec_times[:N-q-1] - vec_times[q+1:N] # change sign - used in likelihood computation

            self.tau_ij_minus_list.append(torch.tensor(minus_tau_ij))


    def check_convergence(self):
        self.max_norm = (self.coeffs - self.coeffs_old).abs().max()
        if self.max_norm < self.tol:
            return True
        self.coeffs_old = self.coeffs.detach().clone()
        return False


    def callback(self, it, acc, b_acc):
        if (it+1) % self.print_every == 0 or it==0:
            print('\riter: {:4d}/{:d} | ELBO: {:.4f} | Test acc: {:.2f}  | Best Test acc: {:.2f} | dx: {:.4f} | time: {:.2f} s' .format(it+1, self.max_iter, self.elbo_history[it], acc, b_acc, self.max_norm, self.train_time))


    def pos_constraint(self, input):
        res = np.zeros_like(input)
        mask = input > 5
        res[mask] = input[mask]
        res[~mask] = np.log(1. + np.exp(input[~mask]))
        return res


    def minus_one_one_constraint_np(self, input):
        return (2. / (1. + np.exp(-input)) - 1.).clip(min=-1+1e-6, max=1-1e-6)


    def compute_acc(self, Q: int=5):
        x_opt = self.coeffs.detach().numpy()

        size_alpha_betas = 1 + self.Z_dim * self.U_dim**2
        size_phis = self.Z_dim * self.U_dim * (self.U_dim - 1)
        #size_deltas = self.Z_dim * self.U_dim

        m_betas_ind = size_alpha_betas
        sigma_betas_ind = 2*size_alpha_betas
        m_phis_ind = sigma_betas_ind + size_phis
        sigma_phis_ind = m_phis_ind + size_phis
        m_omega_all_ind = sigma_phis_ind + (self.U_dim-1) * self.num_teams
        sigma_omega_all_ind = m_omega_all_ind + (self.HA_dim) * self.num_teams
        rho_ind = sigma_omega_all_ind + (self.HA_dim) * self.num_teams
        omegas_all_ind = rho_ind + (self.U_dim-1) * self.num_teams * self.num_gw

        m_betas_alpha_opt = x_opt[:m_betas_ind]
        sigma_betas_alpha_opt = self.pos_constraint(x_opt[m_betas_ind:sigma_betas_ind])
        m_phis_opt = x_opt[sigma_betas_ind:m_phis_ind].reshape(self.Z_dim, self.U_dim, self.U_dim-1)
        #sigma_phis_opt = self.pos_constraint(x_opt[m_phis_ind:sigma_phis_ind]).reshape(self.Z_dim, self.U_dim, self.U_dim-1)

        m_omegas_all = x_opt[sigma_phis_ind:m_omega_all_ind].reshape(self.num_teams, (self.U_dim-1))
        #sigma_omegas_all = self.pos_constraint(x_opt[m_omega_all_ind:sigma_omega_all_ind]).reshape(20, 15)
        #rho_all = self.minus_one_one_constraint_np(x_opt[sigma_omega_all_ind:rho_ind]).reshape(20, 15)
        #omegas_all = x_opt[rho_ind:omegas_all_ind].reshape(20, 29, 37)

        concent_deltas_opt = self.pos_constraint(x_opt[omegas_all_ind:]).reshape(self.Z_dim, self.U_dim)

        mode_alpha = np.exp(m_betas_alpha_opt[-1] - sigma_betas_alpha_opt[-1]**2)
        mode_betas = np.exp(m_betas_alpha_opt[:-1] - sigma_betas_alpha_opt[:-1]**2).reshape(self.Z_dim, self.U_dim, self.U_dim)

        m_phi_plus_omega_all = np.exp(m_phis_opt[:, None, :, :] + m_omegas_all[None, :, None, :]) # Z x teams x M x M-1
        denom_mode_all = m_phi_plus_omega_all.sum(-1) + 1 # Z x teams x M

        deltas = concent_deltas_opt / concent_deltas_opt.sum(1, keepdims=True)

        Gammas = np.zeros((self.Z_dim, self.num_teams, self.U_dim, self.U_dim))
        Gammas[:, :, :, :self.U_dim-1] = m_phi_plus_omega_all / denom_mode_all[:, :, :, None]
        Gammas[:, :, :, -1] = 1. / denom_mode_all

        true_marks_all = []
        predicted_marks_all = []
        for s in range(self.num_seqs_val):
            times_full = self.data_times_val[s]
            marks_full = self.data_marks_val[s]
            zones_full = self.data_zones_val[s]
            teamsID_full = self.data_teamsID_val[s]
            size_N = len(times_full)

            for ii in range(size_N-1):
                times = times_full[:size_N - ii]
                marks = marks_full[:size_N - ii]
                zones = zones_full[:size_N - ii]
                teamsID = teamsID_full[:size_N - ii]

                true_marks_all.append(marks[-1])
                N = times.size
                ar_x = np.arange(N)
                prob_all = np.zeros((N, self.U_dim))
                prob_all[0] = deltas[zones[0]]
                Q_ = min(self.cut_off, N-1)
                Q_mtr = np.zeros((Q_, N-1))

                for u in range(self.U_dim):
                    for q in range(Q_):
                        marks_pred = u * np.ones(N - 1 -q, dtype=np.int32)
                        tij = times[:N-q-1] - times[q+1:N]
                        zones_q = zones[1+q:]
                        teamsID_q = teamsID[1+q:]

                        Q_mtr[q, q:] = Gammas[zones_q, teamsID_q, marks[:-1-q], marks_pred] * np.exp(mode_betas[zones_q, marks[:-1-q], marks_pred] * tij)

                    prob_all[1:, u] = deltas[zones[1:], u] + mode_alpha * Q_mtr.sum(0)
                prob_all /= prob_all.sum(axis=1, keepdims=True)

                predicted_marks_all.append(np.argmax(prob_all, axis=1)[-1])

        true_marks_all, predicted_marks_all = np.array(true_marks_all), np.array(predicted_marks_all)

        return accuracy_score(true_marks_all, predicted_marks_all)


    def log_likelihood(self, betas_alpha, deltas, phis, omegas_all):
        """
        Compute the value of log-likelihood
        """
        #phis = clamp_preserve_gradients(phis, -5.0, 5.0)
        #omegas_all = clamp_preserve_gradients(omegas_all, -5.0, 5.0)

        alpha = betas_alpha[-1]
        beta_mtr = betas_alpha[:-1].reshape(self.Z_dim, self.U_dim, self.U_dim)
        phis_ = phis.reshape(self.Z_dim, self.U_dim, self.U_dim-1)
        # omegas_all: 30 x 47 x 80

        log_lkl_full = 0.
        gameweek = 0
        ind_tally = np.zeros(self.num_teams, dtype=int)
        ind_gw = np.zeros(self.num_teams, dtype=int)
        for s in range(self.num_seqs):
            marks = self.data_marks[s]
            zones = self.data_zones[s]
            teamsID = self.data_teamsID[s]
            minus_tau_ij = self.tau_ij_minus_list[s]
            N = self.num_jumps[s]

            if (s) % 4 == 0 or s == 0:
                team1, team2 = list(set(teamsID))
                #Need to add 1 to gw of teams in current game, except for first gw where it should remain 0:
                ind_tally[team1] += 1
                ind_tally[team2] += 1
                if ind_tally[team1] >= 2:
                    ind_gw[team1] +=1
                if ind_tally[team2] >= 2:
                    ind_gw[team2] += 1
                #Wish to access specific Omegas:
                omegas_final = torch.zeros((self.num_teams, (self.U_dim-1)))
                for i in range(self.num_teams):
                    omegas_final[i] = omegas_all[i, :, ind_gw[i]]

                phi_plus_omega_all = phis_[:, None, :, :] + omegas_final[None, :, None, :] # Z x teams x M x M-1
                exp_phi_plus_omega_all = phi_plus_omega_all.exp()
                denom_all = exp_phi_plus_omega_all.sum(-1) + 1 # Z x teams x M

                Gammas = torch.zeros((self.Z_dim, self.num_teams, self.U_dim, self.U_dim))
                Gammas[:, :, :, :self.U_dim-1] = exp_phi_plus_omega_all / denom_all[:, :, :, None]
                Gammas[:, :, :, -1] = 1. / denom_all


            Q_mtr = torch.zeros((self.cut_off, N-1))
            Q_denom = torch.zeros((N-1, self.cut_off))
            for q in range(self.cut_off):
                tij = minus_tau_ij[q, q:]
                marks_1_q = marks[:-1-q]
                marks_q_1 = marks[1+q:]
                zones_q = zones[1+q:]
                teamsID_q = teamsID[1+q:]

                Q_mtr[q, q:] = Gammas[zones_q, teamsID_q, marks_1_q, marks_q_1] * (beta_mtr[zones_q, marks_1_q, marks_q_1] * tij).exp()
                Q_denom[q:, q] = (Gammas[zones_q, teamsID_q, marks_1_q] * (beta_mtr[zones_q, marks_1_q] * tij[:, None]).exp()).sum(1)

            numer = deltas[zones[0], marks[0]].log() + (deltas[zones[1:], marks[1:]] + alpha * Q_mtr.sum(0)).log().sum()
            denom = (1. + alpha * Q_denom.sum(1)).log().sum()

            log_lkl_full += numer - denom

        return log_lkl_full


    def log_prior(self, omegas_all, m_omegas_all, sigma_omegas_all, rho_all):
        """
        Compute the value of log-prior
        """
        # log-pdf omegas
        stacked_rho = torch.hstack((rho_all, rho_all))[:, :-1] #  20 x 29
        stacked_sigma = torch.hstack((sigma_omegas_all, sigma_omegas_all))[:, :-1] #  20 x 29
        one_rho_sq = 1. - stacked_rho.square() # 20 x 29
        diff = omegas_all - m_omegas_all[:, :, None] # 20 x 29 x 37
        Lq_diff = torch.zeros_like(diff)
        Lq_diff[:, :, -1] = diff[:, :, -1] * one_rho_sq.sqrt()
        Lq_diff[:, :, :-1] = diff[:, :, :-1] - stacked_rho[:, :, None] *  diff[:, :, 1:]
        Lq_diff /= stacked_sigma[:, :, None]
        log_prior = - 0.5 * (Lq_diff.square().sum() + 2 * 2 * self.num_gw * sigma_omegas_all.log().sum() - one_rho_sq.log().sum())

        return log_prior


    def log_joint(self, samples_betas_alpha, samples_deltas, samples_phis, omegas_all, m_omegas_all, sigma_omegas_all, rho_all):
        """
        Compute log joint
        """
        # Compute the log-likelihood
        loglik = self.log_likelihood(samples_betas_alpha, samples_deltas, samples_phis, omegas_all)

        # Compute the log-prior
        logprior = self.log_prior(omegas_all, m_omegas_all, sigma_omegas_all, rho_all)

        return loglik + logprior


    def objective(self, x):
        """
        Importance weighted variational objective function
        Arguments:
        ----------
        x : torch.Tensor
             The variational parameters to be optimized
        """
        # Split the parameters into concentration parameters and means/stds
        m_betas_alpha = x[:self.m_betas_ind]
        sigma_betas_alpha = self.positive_constraint(x[self.m_betas_ind:self.sigma_betas_ind])
        m_phis = x[self.sigma_betas_ind:self.m_phis_ind]
        sigma_phis = self.positive_constraint(x[self.m_phis_ind:self.sigma_phis_ind])

        m_omegas_all = x[self.sigma_phis_ind:self.m_omega_all_ind].reshape(self.num_teams, (self.U_dim-1))
        sigma_omegas_all = self.positive_constraint(x[self.m_omega_all_ind:self.sigma_omega_all_ind]).reshape(self.num_teams, (self.HA_dim))
        rho_all = self.minus_one_one_constraint(x[self.sigma_omega_all_ind:self.rho_ind]).mul(2.).add(-1.).clamp(self.l_b, self.u_b).reshape(self.num_teams, (self.HA_dim))

        omegas_all = x[self.rho_ind:self.omegas_all_ind].reshape(self.num_teams, (self.U_dim-1), self.num_gw)

        concent_deltas = self.positive_constraint(x[self.omegas_all_ind:]).reshape(self.Z_dim, self.U_dim)

        # Sample noise and Dirichlet distributed probability vectors
        samples_betas_alpha = (m_betas_alpha + sigma_betas_alpha * torch.randn(self.num_samples, self.size_alpha_betas)).exp()
        samples_phis = m_phis + sigma_phis * torch.randn(self.num_samples, self.size_phis)

        dirichlet_deltas = torch.distributions.dirichlet.Dirichlet(concent_deltas)
        samples_deltas = dirichlet_deltas.rsample((self.num_samples, )) # num_samples x Z x U

        elbo_value = 0.0
        # Compute a Monte Carlo estimate of ELBO
        for l in range(self.num_samples):
            elbo_value += self.log_joint(samples_betas_alpha[l], samples_deltas[l], samples_phis[l], omegas_all, m_omegas_all, sigma_omegas_all, rho_all)
        elbo_value /= self.num_samples
        return elbo_value


    def fit(self, coeffs_0):
        if not isinstance(coeffs_0, np.ndarray):
            raise TypeError("Invalid 'coeffs_0' provided. Coeffs must be a numpy array")

        if coeffs_0.size != self.n_var_params or coeffs_0.ndim != 1:
            raise ValueError("Invalid size of 'coeffs_0' provided. \
                             'coeffs_0' must be a numpy array of size (" + str(self.n_var_params) + ", )")


        self.coeffs = torch.tensor(coeffs_0, requires_grad=True)
        self.coeffs_old = self.coeffs.detach().clone()

        optimizer = torch.optim.Adam([self.coeffs], lr=self.lr_init)
        scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=self.lr_gamma)
        self.train_time = 0.
        impatient = 0
        patience = 200
        best_acc = 0.
        self.time_of_start = time.time()
        for iter in range(self.max_iter):
            start_t = time.time()
            # Gradient update
            optimizer.zero_grad()
            elbo_val = -1.0 * self.objective(self.coeffs)
            elbo_val.backward()
            optimizer.step()
            scheduler.step()
            self.train_time += time.time() - start_t

            self.elbo_history[iter] = -elbo_val.detach().numpy()
            acc_val = 100 * self.compute_acc()

            if (acc_val - best_acc) < 1e-4:
                impatient += 1
                if acc_val > best_acc:
                    best_acc = acc_val
                    self.best_coeffs = self.coeffs.detach().clone()
            elif acc_val > best_acc:
                best_acc = acc_val
                self.best_coeffs = self.coeffs.detach().clone()
                impatient = 0

            if impatient >= patience:
                print(f'Breaking due to early stopping at epoch {iter}')
                break

            # Check that the optimization did not fail
            if torch.isnan(self.coeffs).any():
                raise ValueError('NaNs in coeffs! Stop optimization...')

            # Convergence check
            if self.check_convergence():
                if self.verbose:
                    self.callback(iter, acc_val, best_acc)
                    print('Converged!')
                break
            elif self.verbose:
                self.callback(iter, acc_val, best_acc)

        print()
        self.time_of_finish = time.time()
        self.total_time_taken = self.time_of_finish - self.time_of_start

def pos_constraint2(input):
    res = np.zeros_like(input)
    mask = input > 5
    res[mask] = input[mask]
    res[~mask] = np.log(1. + np.exp(input[~mask]))
    return res

def minus_one_one_constraint_np(input):
    return (2. / (1. + np.exp(-input)) - 1.).clip(min=-1+1e-6, max=1-1e-6)

def compute_scores(times_list, marks_list, zones_list, teamsID_list, alpha, beta_mtr, delta_vec, Gamma_mtr, Q: int=5):
    S = len(times_list)
    U_dim = delta_vec.shape[1]
    true_marks_all = []
    predicted_marks_all = []


    for s in range(S):
        times_full = times_list[s]
        marks_full = marks_list[s]
        zones_full = zones_list[s]
        teamsID_full = teamsID_list[s]
        size_N = times_full.size

        for ii in range(size_N-1):
            times = times_full[:size_N - ii]
            marks = marks_full[:size_N - ii]
            zones = zones_full[:size_N - ii]
            teamsID = teamsID_full[:size_N - ii]

            true_marks_all.append(marks[-1])
            N = times.size
            ar_x = np.arange(N)
            prob_all = np.zeros((N, U_dim))
            prob_all[0] = delta_vec[zones[0]]
            Q_ = min(Q, N-1)
            Q_mtr = np.zeros((Q_, N-1))

            for u in range(U_dim):
                for q in range(Q_):
                    marks_pred = u * np.ones(N - 1 -q, dtype=np.int32)
                    tij = times[:N-q-1] - times[q+1:N]
                    zones_q = zones[1+q:]
                    teamsID_q = teamsID[1+q:]

                    Q_mtr[q, q:] = Gamma_mtr[zones_q, teamsID_q, marks[:-1-q], marks_pred] * np.exp(beta_mtr[zones_q, marks[:-1-q], marks_pred] * tij)

                prob_all[1:, u] = delta_vec[zones[1:], u] + alpha * Q_mtr.sum(0)
            prob_all /= prob_all.sum(axis=1, keepdims=True)

            predicted_marks_all.append(np.argmax(prob_all, axis=1)[-1])

    true_marks_all, predicted_marks_all = np.array(true_marks_all), np.array(predicted_marks_all)
    assert true_marks_all.size == predicted_marks_all.size
    print("  Num predicted:", predicted_marks_all.size)
    return f1_score(true_marks_all, predicted_marks_all, average='micro'), accuracy_score(true_marks_all, predicted_marks_all)


def lognorm_pdf(x, mu, sigma):
    if isinstance(x, np.ndarray):
        assert (x > 0).all()
    else:
        assert x > 0

    return (np.exp(-(np.log(x) - mu)**2 / (2 * sigma**2)) / (x * sigma * np.sqrt(2 * np.pi)))


def norm_pdf(x, mu, sigma):
    return (np.exp(-(x - mu)**2 / (2 * sigma**2)) / (sigma * np.sqrt(2 * np.pi)))


def norm_logpdf(x, mu, sigma):
    log_exp_term = np.square((x - mu) / sigma)
    log_det = np.log(sigma)
    return -0.5 * (np.log(2 * np.pi) + log_exp_term) - log_det

def lognorm_logpdf(x, mu, sigma):
    log_x = np.log(x)
    return norm_logpdf(log_x, mu, sigma) - log_x


def log_pdf_mixture_lognormals_stable(x, pi_vec, mu_vec, sigma_sq_vec):
    assert len(pi_vec) == len(mu_vec) and len(pi_vec) == len(sigma_sq_vec)
    assert np.isclose(np.sum(pi_vec), 1.) and (pi_vec >= 0.0).all() and (sigma_sq_vec > 0.0).all()
    assert (x >= 0.0).all()
    pi_vec[pi_vec <= 1e-8] = 1e-8

    if isinstance(x, np.ndarray):
        x[x <= 1e-8] = 1e-8
    else:
        x = 1e-8 if x <= 1e-8 else x
    
    sigma_vec = np.sqrt(sigma_sq_vec) 
    log_x = np.log(x)
    
    weighted_log_prob = np.zeros((len(x), len(pi_vec))) if isinstance(x, np.ndarray) else np.zeros((1, len(pi_vec)))
    for component in range(len(pi_vec)):
        weighted_log_prob[:, component] = norm_logpdf(log_x, mu_vec[component], sigma_vec[component])

    weighted_log_prob += np.log(pi_vec)

    return logsumexp(weighted_log_prob, axis=1) - log_x


def load_pkl(name, dict_name):
    with open(name, 'rb') as f:
        data = pickle.load(f, encoding='latin-1')
        num_types = data['dim_process']
        data = data[dict_name]
        
    return data, int(num_types)


def opt_logNormal_mixture(time_gap, event_type, num_types: int, num_components: int=2):
    time_gap = [tg[1:] for tg in time_gap]
    event_type = [et[:-1] for et in event_type]

    inter_arriv_full = np.concatenate(time_gap)
    inter_arriv_full[inter_arriv_full <= 1e-8] = 1e-8
    inter_arriv_full[inter_arriv_full > 150] = 150
    marks_inter_full = np.concatenate(event_type)

    mean_vec_opt = np.zeros(num_types)
    weights_opt = np.zeros((num_types, num_components))
    means_opt = np.zeros((num_types, num_components))
    variances_opt = np.ones((num_types, num_components))
    for u in range(num_types):
        data = inter_arriv_full[marks_inter_full == u][:, None]
        log_data = np.log(data)

        if len(data) > 5:
            #print('inter-arrivals mean: %.4f | var: %.4f | max: %.4f | min: %.15f | #(< 1e-4): %d | #events: %d' %(data.mean(), data.var(), data.max(), data.min(), (data < 1e-4).sum(), len(data)))
            #model = GeneralMixtureModel.from_samples(LogNormalDistribution, n_components=num_components, X=log_data)
            #model.fit(log_data)
            timemodel = GaussianMixture(n_components=num_components, n_init=1, init_params="k-means++", tol=1e-6, max_iter=30)
            timemodel.fit(log_data)

            weights_opt[u] = timemodel.weights_
            means_opt[u] = timemodel.means_.flatten()
            variances_opt[u] = timemodel.covariances_.flatten()
            mean_vec_opt[u] = (weights_opt[u] * np.exp(means_opt[u] + 0.5 * variances_opt[u])).sum()
        else:
            #print("No data!")
            mean_vec_opt[u] = 1e-8
            weights_opt[u, 0] = 1.0
            means_opt[u, 0] = -8 * np.log(10) #np.log(0.2) - 0.5
            
            """
            mean_vec_opt[u] = 0.2
            weights_opt[u] = np.ones(num_components) / num_components
            mean_vec_opt[u] = (weights_opt[u] * np.exp(-1e5 + 0.5 * 1e-8)).sum()
            """

    return mean_vec_opt, weights_opt, means_opt, variances_opt
print('Done')

Done


Below Runs The Mark Model

In [23]:
def RunModel(times,marks,zones,teams,save_as,lr_initial,lr,epochs,q):
    if __name__ == "__main__":
        torch.manual_seed(2105098)
        np.random.seed(2105098)
        
        times_list_train,marks_list_train,teams_list_train,zones_list_train,times_list_val,marks_list_val,teams_list_val,zones_list_val = test_train(times,marks,zones,teams)
        model = VI_Framework(times_list_train, marks_list_train, zones_list_train, teams_list_train, 
                             times_list_val, marks_list_val, zones_list_val, teams_list_val,
                             num_samples=1, max_iter=epochs, interval=200, print_every=5, cut_off=q, verbose=True, lr_gamma = lr,lr_init=lr_initial)
    
        print('                   U:', model.U_dim)
        print('                   Z:', model.Z_dim)
        print('             teamsID:', model.num_teams)
        print('          Train seqs:', len(times_list_train))
        print('     Train num_jumps:', sum(model.num_jumps))
        print('           Test seqs:', len(times_list_val))
        print('      Test num_jumps:', sum(model.num_jumps_val))
        
        x_0 = np.zeros(int(model.n_var_params))
        x_0[:model.m_betas_ind] = np.random.normal(loc=2.1, scale=0.1, size=int(model.size_alpha_betas))
        x_0[model.m_betas_ind:model.sigma_betas_ind] = np.log(np.exp(np.clip(np.random.normal(loc=0.2, scale=0.1, size=int(model.size_alpha_betas)), 1e-1, 2.0)) - 1.)
        x_0[model.sigma_betas_ind:model.m_phis_ind] = np.random.normal(loc=2.1, scale=0.1, size=int(model.size_phis))
        x_0[model.m_phis_ind:model.sigma_phis_ind] = np.log(np.exp(np.clip(np.random.normal(loc=0.2, scale=0.1, size=int(model.size_phis)), 1e-1, 2.0)) - 1.)
    
        #NEED TO EDIT BELOW:
        #UDIM = 48
        #NUM TEAMS = 30
        #Num Games = 1230 
        #Num Gameweeks: 82 
        #Playoff Gamweeeks: Up to 4x7=28 - WILL IGNORE PLAYOFFS FOR NOW
        
        x_0[model.sigma_phis_ind:model.m_omega_all_ind] = np.random.normal(loc=0.5, scale=0.1, size= int(model.U_dim-1)*int(model.num_teams))
        x_0[model.m_omega_all_ind:model.sigma_omega_all_ind] = np.random.normal(loc=1.1, scale=0.1, size=int(model.num_teams*(model.HA_dim)))
        x_0[model.sigma_omega_all_ind:model.rho_ind] = np.random.normal(loc=0, scale=0.1, size=int(model.num_teams*(model.HA_dim)))
        x_0[model.rho_ind:model.omegas_all_ind] = np.random.normal(loc=0, scale=0.1, size=int((model.U_dim-1)*model.num_teams*model.num_gw))
        x_0[model.omegas_all_ind:] = np.log(np.exp(np.clip(np.random.normal(loc=0.2, scale=0.1, size=int(model.size_deltas)), 1e-1, 2.0)) - 1.)
    
        print('Number of parameters:', x_0.size)
        print("\nStart training Marks...")
        model.fit(x_0)
        print("Done!")
        print('Training time: %.4f s' %model.train_time)
        print('Total Time To Run:',model.total_time_taken)
        x_opt = model.best_coeffs.detach().numpy()
        np.save(f'D:/Uni Work/Year 4/Diss/Diss_R/ProjectDiss/{save_as}.npy', x_opt)
        np.save(f'C:/Users/joshh/GitFolder/NBAScalableMPP/{save_as}.npy', x_opt)
        
        print("\nStart Mark Assessment...")
        print(20 * '-' + '\n            Dataset: NBA Play-By-Play' )
    
        #Below computes Mark Scores
        #Seperating Parameters:
        m_betas_alpha_opt = x_opt[:model.m_betas_ind]
        sigma_betas_alpha_opt = pos_constraint2(x_opt[model.m_betas_ind:model.sigma_betas_ind])
        m_phis_opt = x_opt[model.sigma_betas_ind:model.m_phis_ind].reshape(model.Z_dim, model.U_dim, model.U_dim-1)
        sigma_phis_opt = pos_constraint2(x_opt[model.m_phis_ind:model.sigma_phis_ind]).reshape(model.Z_dim, model.U_dim, model.U_dim-1)
        
        m_omegas_all = x_opt[model.sigma_phis_ind:model.m_omega_all_ind].reshape(model.num_teams, (model.U_dim-1))
        sigma_omegas_all = pos_constraint2(x_opt[model.m_omega_all_ind:model.sigma_omega_all_ind]).reshape(model.num_teams, (model.HA_dim))
        rho_all = minus_one_one_constraint_np(x_opt[model.sigma_omega_all_ind:model.rho_ind]).reshape(model.num_teams, (model.HA_dim))
    
        omegas_all = x_opt[model.rho_ind:model.omegas_all_ind].reshape(model.num_teams, (model.U_dim-1), model.num_gw)
    
        concent_deltas_opt = pos_constraint2(x_opt[model.omegas_all_ind:]).reshape(model.Z_dim, model.U_dim)
    
        mode_alpha = np.exp(m_betas_alpha_opt[-1] - sigma_betas_alpha_opt[-1]**2)
        mode_betas = np.exp(m_betas_alpha_opt[:-1] - sigma_betas_alpha_opt[:-1]**2).reshape(model.Z_dim, model.U_dim, model.U_dim)
    
        m_phi_plus_omega_all = np.exp(m_phis_opt[:, None, :, :] + m_omegas_all[None, :, None, :]) # Z x teams x M x M-1
        denom_mode_all = m_phi_plus_omega_all.sum(-1) + 1 # Z x teams x M
    
        deltas = concent_deltas_opt / concent_deltas_opt.sum(1, keepdims=True)
    
        Gammas = np.zeros((model.Z_dim, model.num_teams, model.U_dim, model.U_dim))
        Gammas[:, :, :, :model.U_dim-1] = m_phi_plus_omega_all / denom_mode_all[:, :, :, None]
        Gammas[:, :, :, -1] = 1. / denom_mode_all
    
        f1_sc, acc_sc = compute_scores(times_list_val, marks_list_val, zones_list_val, teams_list_val, mode_alpha, mode_betas, deltas, Gammas, Q=5)
    
        print('Test F1: {:.2f}%  | Test Acc: {:.2f}%' .format(100 * f1_sc, 100 * acc_sc))
    
        return model

Below Runs the Time Component

In [25]:
def TimeTrain(times,marks,zones,teams, num_comp, U_dim):
    
        times_list_train,marks_list_train,teams_list_train,zones_list_train,times_list_val,marks_list_val,teams_list_val,zones_list_val = test_train(times,marks,zones,teams)
        #Below Computes Time Scores
        #Z_dim, num_types, num_teams = model_ttest.Z_dim, 40, model_ttest.num_teams
        Z_dim, num_types, num_teams = 2, U_dim, 30
        print(num_types)
        time_gap_train = [np.concatenate(([seq[0]], seq[1:] - seq[:-1])) for seq in times_list_train]
        time_gap_test = [np.concatenate(([seq[0]], seq[1:] - seq[:-1])) for seq in times_list_val]
    
        assert sum(len(seq) for seq in time_gap_train) == sum(len(seq) for seq in marks_list_train)
    
        num_events_train = sum(len(sq) for sq in times_list_train)
        num_events_test = sum(len(sq) for sq in times_list_val)
    
        #print(20 * '-' + '\n            Dataset: NBA Play-By-Play' )
        #print('                  U:', num_types)
        #print('   num_events train:', num_events_train)
        #print('    num_events test:', num_events_test)
        #print('num sequences train:', len(times_list_train))
        #print(' num sequences test:', len(times_list_val))
    
        inter_arriv_full = np.concatenate(time_gap_train + time_gap_test)
        mask_zeros = inter_arriv_full <= 1e-8
        inter_arriv_full[mask_zeros] = 1e-7
        #inter_arriv_full = inter_arriv_full[~mask_zeros]
        #print("%% inter-arrival < 1e-8: %.4f" %(100*mask_zeros.mean()))

        clip_upper = inter_arriv_full >= 150
        inter_arriv_full[clip_upper] = 150
        #inter_arriv_full = inter_arriv_full[~clip_upper]
        #print("%% inter-arrival > 150: %.4f" %(100*clip_upper.mean()))

    
        #print("Start training Times...")
        start_t = time.time()
        log_first_gap_tr = np.log(np.array([time_gap_seq[0] + 1e-8 for time_gap_seq in time_gap_train]))
        mu_hat = log_first_gap_tr.mean()
        sigma_hat = log_first_gap_tr.std() + 1e-5
    
        mean_vec_opt, weights_opt, means_opt, variances_opt = opt_logNormal_mixture(time_gap_train, marks_list_train, num_types=num_types, num_components=num_comp)   
        print(mean_vec_opt)
        train_time = time.time() - start_t
        #print("Done!")
    
        log_lkl_times, se_mean, se_naive = 0., 0., 0.
        num_pred = 0
        for s in range(len(times_list_val)):
            times = times_list_val[s]
            marks = marks_list_val[s]
            time_gaps = time_gap_test[s]
            time_gaps[time_gaps <= 1e-8] = 1e-8
            num_pred += marks.size - 1            
            for idx in range(1, len(marks)):
                true_tau = time_gaps[idx]
                se_mean += (true_tau - mean_vec_opt[marks[idx-1]])**2
                se_naive += (true_tau - time_gaps[idx-1])**2
            log_inc = lognorm_logpdf(time_gaps[0], mu_hat, sigma_hat)
            if np.isnan(log_inc):
                log_inc = log_past
                print("1")
            log_lkl_times += log_inc
            log_past = log_inc
            
            time_gaps = time_gaps[1:]
            marks = marks[:-1]
            for u in range(num_types):
                data = time_gaps[marks == u]
                log_inc = log_pdf_mixture_lognormals_stable(data, weights_opt[u], means_opt[u], variances_opt[u]).sum()
                log_lkl_times += log_inc
        log_lkl_times /= num_events_test
        #assert jumps_tst == (jumps_tst_se + len(first_time_gaps_test))
        
        rmse_mean = np.sqrt(se_mean / num_events_test)
        rmse_naive = np.sqrt(se_naive / num_events_test)
    
        #print("  Num predicted:", num_pred)
        #print("  Training time: %.1f min" %(train_time / 60.))
        print('   Test log-lkl: %.4f nats' %log_lkl_times)
        print('      Test RMSE: %.4f' %rmse_mean)
        print('Test RMSE naive: %.4f' %rmse_naive)
        return rmse_mean, log_lkl_times

The following runs all of the Mark models:

In [ ]:
#With Rebounds
model_2022 = RunModel(times_all,marks_all,zones_all,teams_all, '2022x0_test',lr_initial=0.05,lr=0.995,epochs=1000,q=1)
model_2022_2 = RunModel(times_all,marks_all,zones_all,teams_all, '2022x0_200',lr_initial=0.05,lr=0.995,epochs=200,q=1)
model_2023 = RunModel(times_all_2023,marks_all_2023,zones_all_2023,teams_all_2023, '2023x0_test',lr_initial=0.05,lr=0.995,epochs=1000,q=1)
model_2023_2 = RunModel(times_all_2023,marks_all_2023,zones_all_2023,teams_all_2023, '2023x0_200',lr_initial=0.05,lr=0.995,epochs=200,q=1)

model_2022_q2 = RunModel(times_all,marks_all,zones_all,teams_all, '2022x0_q2',lr=0.99,epochs=200,q=2, lr_initial = 0.05)
model_2022_q3 = RunModel(times_all,marks_all,zones_all,teams_all, '2022x0_q3',lr=0.99,epochs=200,q=3, lr_initial = 0.05)
model_2022_q4 = RunModel(times_all,marks_all,zones_all,teams_all, '2022x0_q4',lr=0.99,epochs=200,q=4, lr_initial = 0.05)
model_2022_q5 = RunModel(times_all,marks_all,zones_all,teams_all, '2022x0_q5',lr=0.99,epochs=200,q=5, lr_initial = 0.05)
model_2022_q10 = RunModel(times_all,marks_all,zones_all,teams_all, '2022x0_q10',lr=0.99,epochs=200,q=10, lr_initial = 0.05)

model_2023_q2 = RunModel(times_all_2023,marks_all_2023,zones_all_2023,teams_all_2023, '2023x0_q2',lr=0.99,epochs=200,q=2, lr_initial = 0.05)
model_2023_q3 = RunModel(times_all_2023,marks_all_2023,zones_all_2023,teams_all_2023, '2023x0_q3',lr=0.99,epochs=200,q=3, lr_initial = 0.05)
model_2023_q4 = RunModel(times_all_2023,marks_all_2023,zones_all_2023,teams_all_2023, '2023x0_q4',lr=0.99,epochs=200,q=4, lr_initial = 0.05)
model_2023_q5 = RunModel(times_all_2023,marks_all_2023,zones_all_2023,teams_all_2023, '2023x0_q5',lr=0.99,epochs=200,q=5, lr_initial = 0.05)
model_2023_q10 = RunModel(times_all_2023,marks_all_2023,zones_all_2023,teams_all_2023, '2023x0_q10',lr=0.99,epochs=200,q=10, lr_initial = 0.05)

#Without Rebounds
model_2022_noreb = RunModel(noreb_times_all_2022,noreb_marks_all_2022,noreb_zones_all_2022,noreb_teams_all_2022, '2022x0_noreb',lr_initial=0.05,lr=0.99,epochs=200,q=1)
model_2023_noreb = RunModel(noreb_times_all_2023,noreb_marks_all_2023,noreb_zones_all_2023,noreb_teams_all_2023, '2023x0_noreb',lr_initial=0.05,lr=0.99,epochs=200,q=1)

model_2023_noreb_q2 = RunModel(noreb_times_all_2022,noreb_marks_all_2022,noreb_zones_all_2022,noreb_teams_all_2022, '2022x0_noreb_q2',lr_initial=0.05,lr=0.99,epochs=250,q=2)
model_2023_noreb_q3 = RunModel(noreb_times_all_2022,noreb_marks_all_2022,noreb_zones_all_2022,noreb_teams_all_2022, '2022x0_noreb_q3',lr_initial=0.05,lr=0.99,epochs=250,q=3)


model_2023_noreb_q2 = RunModel(noreb_times_all_2023,noreb_marks_all_2023,noreb_zones_all_2023,noreb_teams_all_2023, '2023x0_noreb_q2',lr_initial=0.05,lr=0.99,epochs=250,q=2)
model_2023_noreb_q3 = RunModel(noreb_times_all_2023,noreb_marks_all_2023,noreb_zones_all_2023,noreb_teams_all_2023, '2023x0_noreb_q3',lr_initial=0.05,lr=0.99,epochs=250,q=3)

In [ ]:
TimeTrain(times_all,marks_all,zones_all,teams_all,1,42)
TimeTrain(times_all,marks_all,zones_all,teams_all,2,42)
TimeTrain(times_all,marks_all,zones_all,teams_all,3,42)
TimeTrain(times_all,marks_all,zones_all,teams_all,4,40)

TimeTrain(times_all_2023,marks_all_2023,zones_all_2023,teams_all_2023,1,42)
TimeTrain(times_all_2023,marks_all_2023,zones_all_2023,teams_all_2023,2,42)
TimeTrain(times_all_2023,marks_all_2023,zones_all_2023,teams_all_2023,3,42)
TimeTrain(times_all_2023,marks_all_2023,zones_all_2023,teams_all_2023,4,42)

TimeTrain(noreb_times_all_2023,noreb_marks_all_2023,noreb_zones_all_2023,noreb_teams_all_2023,1,40)
TimeTrain(noreb_times_all_2023,noreb_marks_all_2023,noreb_zones_all_2023,noreb_teams_all_2023,2,40)
TimeTrain(noreb_times_all_2023,noreb_marks_all_2023,noreb_zones_all_2023,noreb_teams_all_2023,3,40)
TimeTrain(noreb_times_all_2023,noreb_marks_all_2023,noreb_zones_all_2023,noreb_teams_all_2023,4,40)

TimeTrain(noreb_times_all_2022,noreb_marks_all_2022,noreb_zones_all_2022,noreb_teams_all_2022,1,40)
TimeTrain(noreb_times_all_2022,noreb_marks_all_2022,noreb_zones_all_2022,noreb_teams_all_2022,2,40)
TimeTrain(noreb_times_all_2022,noreb_marks_all_2022,noreb_zones_all_2022,noreb_teams_all_2022,3,40)
TimeTrain(noreb_times_all_2022,noreb_marks_all_2022,noreb_zones_all_2022,noreb_teams_all_2022,4,40)